In [1]:
import os
import openai
from sklearn import metrics
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import nltk
import re
import pickle
import pandas as pd
import numpy as np
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import helper.sklearn_to_csv
import jsonlines

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
report = metrics.classification_report(['+','-'], ['+', 'o'])
print(report)

              precision    recall  f1-score   support

           +       1.00      1.00      1.00         1
           -       0.00      0.00      0.00         1
           o       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

In [9]:
COL_OPINION = 0
COL_SPEECH = 8

RUNS = 1
TEST_SPLIT = 0.15
SHUFFLE = True
STRATIFY = True
CLASS_WEIGHTS = True
REMOVE_STOP_WORDS = False
STEMMING = False
LOWERING = False
NO_PUNCTUATION = False

OUT_NAME = f'measures_gpt3_01_{TEST_SPLIT}_Shuff{SHUFFLE}_Strat{STRATIFY}_Weigh{CLASS_WEIGHTS}_NoStop{REMOVE_STOP_WORDS}_Stem{STEMMING}_Low{LOWERING}_NoPun{NO_PUNCTUATION}'
PATH_METRICS = f'../results/{OUT_NAME}.csv'
PATH_DATASET = "../protocols/labelled/massnahmen_relevant.csv"
PATH_PERFORMANCE_GRAPH = f'../results/{OUT_NAME}.png'

In [36]:
# read data from csv
categories = ['-', 'o', '+']
data = pd.read_csv(PATH_DATASET, sep='\t', header=None, skiprows=[0])
data["opinion_integer"] = 0
data.loc[data[COL_OPINION] == '-', "opinion_integer"] = 0
data.loc[data[COL_OPINION] == 'o', "opinion_integer"] = 1
data.loc[data[COL_OPINION] == '+', "opinion_integer"] = 2
data["comment_clean"] = data.loc[data[COL_SPEECH].str.len() > 1, COL_SPEECH]
#data = data[["comment_clean", "opinion_integer"]]
#data.columns = ["text", "label"]
data.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,opinion_integer,comment_clean
0,+,False,172479,2021_02_24,85,ÖVP,Abgeordnete MMMag. Gertraud Salzmann (ÖVP),True,"Die Tourismusleistungen sind derartig gut, das...",False,False,False,True,False,2,"Die Tourismusleistungen sind derartig gut, das..."
1,-,False,174259,2021_02_24,85,ÖVP,Abgeordnete Gabriela Schwarz (ÖVP),True,"Da ist nämlich schon die Rede davon, ob sich j...",True,False,False,False,False,0,"Da ist nämlich schon die Rede davon, ob sich j..."


In [18]:
train_data, test_data, train_label, test_label = train_test_split(data["text"], data["label"], test_size=TEST_SPLIT, shuffle=SHUFFLE, stratify=data["label"])

In [38]:
train_data1, test_data1 = train_test_split(data, test_size=TEST_SPLIT, shuffle=SHUFFLE, stratify=data["opinion_integer"])


In [41]:
train_data1.to_csv('openai/train.csv', sep='\t')
test_data1.to_csv('openai/test.csv', sep='\t')

In [19]:
train = pd.read_csv('openai/train.csv', sep='\t')
train = train[['comment_clean', 'opinion_integer']]
train.columns = ['text', 'label']
test = pd.read_csv('openai/test.csv', sep='\t')
#test = test[['comment_clean', 'opinion_integer']]
#test.columns = ['text', 'label']
train_data = train['text']
train_label = train['label']
#test_data = test['text']
#test_label = test['label']

In [13]:
train_data_ = train_data.tolist()
train_label_ = train_label.tolist()

In [14]:
json_entries = []
for i in range(len(train_data)):
    json_entry = {"text": train_data_[i], "label": str(train_label_[i])}
    json_entries.append(json_entry)

In [15]:
with jsonlines.open('openai/train.jsonl', 'w') as f:
    for json_entry in json_entries:
        f.write(json_entry)

In [17]:
openai.File.create(file=open("openai/train.jsonl"), purpose="classifications")

Upload progress: 100%|██████████| 1.55M/1.55M [00:04<00:00, 343kit/s] 


<File file id=file-LY7xgPrX3T1mukP2qWTg12zI at 0x7fcab6a9a220> JSON: {
  "bytes": 1549370,
  "created_at": 1622642561,
  "filename": "openai/train.jsonl",
  "id": "file-LY7xgPrX3T1mukP2qWTg12zI",
  "object": "file",
  "purpose": "classifications",
  "status": "uploaded",
  "status_details": null
}

In [32]:
result = openai.Classification.create(
    file="file-LY7xgPrX3T1mukP2qWTg12zI",
    query="Die Situation in den Lagern ist aufgrund der Covid-19-Pandemie allerdings nicht besser geworden, ganz im Gegenteil, sie spitzt sich zu einer humanitären und gesundheitli­chen Katastrophe zu. Allein im Lager Moria auf Lesbos leben 20 000 Menschen zu­sammengepfercht auf engstem Raum unter widrigsten Verhältnissen: Die Wasserver­sorgung ist nicht oder kaum gegeben, 100 Menschen teilen sich eine Toilette, Mas­kenpflicht und Abstandsregeln, wie wir sie kennen, sind in den Camps nicht möglich, Voraussetzungen zum Händewaschen schon gar nicht gegeben. Die Menschen sind aufgrund der Situation generell geschwächt, es ist also wichtig, gerade in der Corona­krise ein Zeichen zu setzen, damit sich das Coronavirus in den Flüchtlingslagern nicht wie ein Lauffeuer ausbreiten kann.",
    search_model="ada",
    model="curie",
    max_examples=3
)

In [33]:
print(result)

{
  "completion": "cmpl-36L9dWILCoakVftK3ERVbkpveCfUo",
  "file": "file-mWnFW3Sb77WYoGohIZVP1u80",
  "label": "1",
  "model": "curie:2020-05-03",
  "object": "classification",
  "search_model": "ada:2020-05-03",
  "selected_examples": [
    {
      "document": 0,
      "label": "1",
      "object": "search_result",
      "score": 6.579,
      "text": "In dieser Zeit aber, in der wir doch R\u00fcckmeldungen bekommen haben, egal ob es per E-Mail, telefonisch, beim Einkaufen im Supermarkt oder bei pers\u00f6nlichen Gespr\u00e4chen war, hat es Menschen gegeben, die in ganz schwierigen Zeiten f\u00fcr uns da waren, und ganz viele dieser Menschen waren in klassischen, typi\u00adschen Frauenberufen, hinsichtlich derer wir jetzt nicht werten sollen, etwa jene, die im Supermarkt gestanden sind. Das sind doch Eindr\u00fccke und Bilder von Menschen, die wir alle haben\u00a0\u2013 etwa wenn wir daran denken, wie wir in den ersten Tagen in den Supermarkt gegangen sind, wo alle Angst gehabt und sich

In [27]:
COL_COMMENT_CLEAN = 'comment_clean'
COL_OPINION_INT = 'opinion_integer'
COL_SENTENCE_ID = '2'

test[COL_SENTENCE_ID]

0      152305
1      193791
2      194861
3       45092
4      166483
        ...  
761    185308
762    106090
763    180019
764    191355
765    159959
Name: 2, Length: 766, dtype: int64

In [35]:
with open('openai/results.csv', 'a+') as f, open('openai/debug.txt', 'a+') as dbg:
    for index, row in test.iterrows():
        print(f"processing #{index}")
        sentence_id = row[COL_SENTENCE_ID]
        actual_label = row[COL_OPINION_INT]
        speech = row[COL_COMMENT_CLEAN]

        print(sentence_id, speech, actual_label)

        result = openai.Classification.create(
            file="file-LY7xgPrX3T1mukP2qWTg12zI",
            query=speech,
            search_model="ada",
            model="curie",
            labels=['0','1','2'],
            max_examples=200
        )
        dbg.write(str(result))
        dbg.write('\n')
        dbg.flush()
        prediction = result['label']

        line = f"{sentence_id}\t{actual_label}\t{prediction}\t{speech}\n"
        f.write(line)
        f.flush()

processing #0
152305 Zu den Testungen: Die Flächenscreenings sind gerade in Umsetzung, auch dieses Wo­chenende wieder, ich selber werde am Sonntag sechs Stunden beim Erfassen dieser Screenings mithelfen. 2
processing #1
193791 Können Sie sich das vorstellen? Wenn 130 000 Dosen am Tag in Österreich verimpft werden müssen, dann schafft alleine das Austria Center, das Vienna International Center, 30 000. – Großartig! 2
processing #2
194861 Die Grenze, bis zu der für Entschädigungen in den Teststraßen keine Sozialversicherungsbeiträge anfallen, konnte von 537 Euro auf 1 000 Euro monatlich angehoben werden. Diese Novelle gilt jetzt auch für jene Freiwilligen, die in den Impfstraßen arbeiten, und darüber bin ich sehr glücklich. Das gilt rückwirkend ab dem 1. 2
processing #3
45092 Das Projekt müsste man jetzt in Angriff nehmen, damit die Kiste läuft, wenn die Impfung dann einmal da ist. 2
processing #4
166483 Sie ändern mit diesem Gesetzesantrag, den Sie hier vorliegen haben, das Medizinprodu

RateLimitError: you exceeded your current quota, please check your plan and billing details

In [44]:
results = pd.read_csv('openai/results.csv', sep='\t', names=['sent_id', 'act_class', 'pred_class', 'speech'])

actual = results['act_class'].tolist()
predictions = results['pred_class'].tolist()

report = metrics.classification_report(actual, predictions)
print(report)

              precision    recall  f1-score   support

           0       0.51      0.82      0.63        78
           1       0.30      0.52      0.38        50
           2       0.84      0.31      0.46       134

    accuracy                           0.50       262
   macro avg       0.55      0.55      0.49       262
weighted avg       0.64      0.50      0.49       262

